In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns

from matplotlib import pyplot as plt
from medcat.cat import CAT
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

C:\Users\BILAL\AppData\Roaming\Python\Python37\site-packages\medcat\cat.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BILAL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
import spacy
spacy.__version__

'3.1.3'

In [33]:
import torch-vision

SyntaxError: invalid syntax (860621229.py, line 1)

In [32]:
transformers.__version__

'4.30.2'

In [28]:
import py2neo
py2neo.__version__

'2021.2.3'

In [3]:
# Load model pack and Create CAT - the main class from medcat used for concept annotation

model_pack_path = 'mc_modelpack_snomed_int_16_mar_2022_25be3857ba34bdd5'
cat = CAT.load_model_pack(model_pack_path)

Found an existing unziped model pack at: mc_modelpack_snomed_int_16_mar_2022_25be3857ba34bdd5, the provided zip will not be touched.
{
  "Model ID": "25be3857ba34bdd5",
  "Last Modifed On": "16 March 2022",
  "History (from least to most recent)": [
    "a474096eb4566638",
    "009617d7ff372682"
  ],
  "Description": "SNOMED INT enriched with UMLS and trained unsupervised on MIMIC-III",
  "Source Ontology": "SnomedCT_InternationalRF2_PRODUCTION_20210131T120000Z",
  "Location": "MedCAT Rosalind machine, available for public download from https://github.com/CogStack/MedCAT",
  "MetaCAT models": {
    "Status": "Detects is a concept Affirmed or Negated/Hypothetical"
  },
  "Basic CDB Stats": {
    "Number of concepts": 354448,
    "Number of names": 2049216,
    "Number of concepts that received training": 29674,
    "Number of seen training examples in total": 20585988,
    "Average training examples per concept": 693.7382220125362
  },
  "Performance": {
    "ner": {},
    "meta": {}
  

In [4]:
import sys
print(sys.version)

3.7.16 (default, Jan 17 2023, 16:06:28) [MSC v.1916 64 bit (AMD64)]


In [5]:
example = ' \nName:  ___                 Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   M\n \nService: ORTHOPAEDICS\n \nAllergies: \nerythromycin base\n \nAttending: ___.\n \nChief Complaint:\nLeft ankle fracture\n \nMajor Surgical or Invasive Procedure:\nORIF left ankle ___\n \nHistory of Present Illness:\n___ yr old man fall down 3 stairs while holding lumbar at \nconstruction\njob, resulting in immediate L ankle pain. ED ankle fracture, \nsurgery performed\n\n \nPast Medical History:\nPrevious R ankle fracture\nRLE cellulitis\nOsteoarthritis\n\n \nSocial History:\n___\nFamily History:\nN/A\n \nPhysical Exam:\nAOXO3\nsplint intact, positive CSM\nPain well controlled, no N/V\n\n \nPertinent Results:\n___ 08:28PM   WBC-9.5 RBC-4.49* HGB-13.7 HCT-41.0 MCV-91 \nMCH-30.5 MCHC-33.4 RDW-13.8 RDWSD-46.7*\n___ 08:28PM   GLUCOSE-96 UREA N-13 CREAT-0.7 SODIUM-145 \nPOTASSIUM-4.1 CHLORIDE-108 TOTAL CO2-24 ANION GAP-13\n___ 08:28PM   CALCIUM-9.0 PHOSPHATE-3.5 MAGNESIUM-1.9\n \nBrief Hospital Course:\nTaken to operating room and underwent surgical fixation of ankle \nfracture on ___. No complications. Post op doing well, pain \nwell controlled. Cleared by ___ to go home without services\n \nMedications on Admission:\nThe Preadmission Medication list is accurate and complete.\n1. Multivitamins 1 TAB PO DAILY \n2. Naproxen 500 mg PO Q8H:PRN Pain - Mild \n\n \nDischarge Medications:\n1.  Acetaminophen 1000 mg PO Q8H \nRX *acetaminophen 500 mg 2 tablet(s) by mouth every eight (8) \nhours Disp #*40 Tablet Refills:*0 \n2.  Aspirin 325 mg PO DAILY \nRX *aspirin 325 mg 1 tablet(s) by mouth once a day Disp #*14 \nTablet Refills:*0 \n3.  Docusate Sodium 100 mg PO BID \nRX *docusate sodium 100 mg 1 capsule(s) by mouth twice a day \nDisp #*30 Capsule Refills:*0 \n4.  Ondansetron 4 mg PO Q8H:PRN severe nausea \nRX *ondansetron 4 mg 1 tablet(s) by mouth every six (6) hours \nDisp #*8 Tablet Refills:*0 \n5.  OxyCODONE (Immediate Release) ___ mg PO Q4H:PRN Pain - \nModerate \nRX *oxycodone 5 mg ___ tablet(s) by mouth every four (4) hours- \n(6) hours Disp #*60 Tablet Refills:*0 \n6.  Multivitamins 1 TAB PO DAILY  \n\n \nDischarge Disposition:\nHome\n \nDischarge Diagnosis:\nleft ankle fracture, s/p ORIF ___\n \nDischarge Condition:\nMental Status: Clear and coherent.\nLevel of Consciousness: Alert and interactive.\nActivity Status: Ambulatory - Independent.\n \nDischarge Instructions:\nNWB to LLE\nKeep splint dry, in place\nElevate LLE\nFollow up in ___ clinic in 2 weeks\n \nFollowup Instructions:\n___\n'

In [6]:
medical_stopwords = stopwords.words("english")


medical_stopwords.extend(['speaking', 'none', 'time', 'flush'])

def process_clinical_note(clinical_note):
    # Define the sections to remove
    sections_to_remove = [
        "Name:",
        "Unit No:",
        "Admission Date:",
        "Discharge Date:",
        "Date of Birth:",
        "Sex:",
        "Service:",
        "Allergies:",
        "Attending:",
        "Past Medical History:",
        "Social History:",
        "Family History:",
        "Vitals:",
        "Pertinent Results:",
        "Medications on Admission:",
        "Discharge Medications:",
        "Discharge Disposition:",
        "Discharge Condition:",
        "Discharge Instructions:",
        "Followup Instructions:"
    ]

    # Split the clinical note into lines
    lines = clinical_note.split('\n')

    # Initialize the processed note
    processed_note = []

    # Flag to exclude lines within unwanted sections
    exclude_section = False

    # Iterate through the lines and filter unwanted sections
    for line in lines:
        if any(section in line for section in sections_to_remove):
            exclude_section = True
        elif line.strip() == "":
            # Empty lines separate sections, so reset the flag
            exclude_section = False

        if not exclude_section:
            processed_note.append(line)

    # Join the lines to create the final note
    final_note = '\n '.join(processed_note)
    
    sections_to_remove = [
        r'chief complaint',
        r'history of present illness',
        r'Major Surgical or Invasive Procedure',
        r'physical exam',
        r'brief hospital course',
        r'Discharge',
        
        r'completed by',
    ]
    
    for pattern in sections_to_remove:
        final_note = re.sub(pattern, '', final_note, flags=re.IGNORECASE)

    # Define patterns to identify negations
    negation_patterns = [
        r'no\s+\w+',
        r'not\s+\w+',
        r'did\s+not\s+have\s+\w+'
    ]
    
    # Filter out sentences with negations
    sentences = [sentence for sentence in final_note.split('\n') if not any(re.search(pattern, sentence, re.IGNORECASE) for pattern in negation_patterns)]

    # Remove keys and special characters
    cleaned_note = re.sub(r'\w+:', '', '\n'.join(sentences), flags=re.IGNORECASE)  # Remove keys (case-insensitive)
    cleaned_note = re.sub(r'[^a-zA-Z\s]', '', cleaned_note)  # Remove special characters
    # Tokenize the note into sentences based on '\n'
    sentences = [sentence.strip() for sentence in cleaned_note.split('\n') if sentence.strip()]

    # Remove stop words and empty sentences
    sentences = [
        ' '.join(word for word in sentence.split() if word.lower() not in medical_stopwords)
        for sentence in sentences
    ]
    sentences = [item for item in sentences if item != '']
    
    final_text = '. '.join(sentence for sentence in sentences)
    return final_text

In [23]:
processed_example = process_clinical_note(example)

In [27]:
icd10_data = {
    'hadm_id': [],
    'icd10_codes': [],
    'text': []
}



In [28]:
# Get the entities from the text
all_entities = cat.get_entities(processed_example)

icd_codes = set()  # Using a set to automatically remove duplicates

for entity_id, entity_data in all_entities['entities'].items():
    icd10 = entity_data.get('icd10', [])
    for code in icd10:
        if code:  # Check if the code is not an empty string
            icd_codes.add(code)

icd10_data['hadm_id'].append('dummy_example')
icd10_data['icd10_codes'].append(list(icd_codes))
icd10_data['text'].append(processed_example)

In [29]:
# Create a DataFrame from the processed data
df_icd10 = pd.DataFrame(icd10_data)
df_icd10

,hadm_id,icd10_codes,text
0,dummy_example,"[S82.80, M79.67]",Left ankle fracture. ORIF left ankle. yr old m...


In [30]:
def reformat(code):
    code = ''.join(code.split('.'))
    code = code[:3] + '.'
    return code

In [31]:
df_icd10['icd10_codes'] = df_icd10['icd10_codes'].apply(lambda lst: [reformat(x) for x in lst])
df_icd10

,hadm_id,icd10_codes,text
0,dummy_example,"[S82., M79.]",Left ankle fracture. ORIF left ankle. yr old m...
